In [1]:
from pathlib import Path
import czifile
import nd2
import tifffile
import napari
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pyclesperanto_prototype as cle
from utils import list_images, read_image, segment_nuclei_2d, segment_marker_positive_nuclei

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (1 refs)>

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [2]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/test_data")

# Construct ROI path from directory_path above
roi_path = directory_path / "ROIs"

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 'raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

In [3]:
# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

# Define your nuclei diameter, it speeds up nuclei detection, if unknown leave it as None
cellpose_nuclei_diameter = None

# Define the amount of blur applied to nuclei
# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 0

In [4]:
# List of subfolder names
roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]
print(f"The following regions of interest will be analyzed: {roi_names}")

The following regions of interest will be analyzed: ['CA', 'DG']


In [ ]:
for image in tqdm(images):

    # Generate maximum intensity projection and extract filename
    img_mip, filename = read_image(image, slicing_factor)

